<a href="https://colab.research.google.com/github/jsanz/elastic_esql_lab/blob/main/02-esql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ES|QL concepts

## Resources

* [Blog post announcement](https://www.elastic.co/blog/esql-elasticsearch-piped-query-language)
* [Documentation](https://www.elastic.co/docs/explore-analyze/query-filter/languages/esql)
* [Reference](https://www.elastic.co/docs/reference/query-languages/esql)
* Webinar: [ES|QL: Search. Aggregate. Transform. Visualize. All with one query](https://www.elastic.co/virtual-events/cd-esql-search-aggregate-transform-visualize-all-with-one-query)
* Technical blog posts in [Search Labs](https://www.elastic.co/search-labs/blog/category/esql)
* [Run this notebook in Google Colaboratory](https://colab.research.google.com/github/jsanz/foss4g_europe_lab/blob/main/02-esql.ipynb)

## Setup

In [1]:
# Install required dependencies using uv, if available, or directly with pip
!pip install -qU elasticsearch geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 929.5/929.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 2.8 MB/s eta 0:00:00


Start with the necessary imports, a couple tweaks, and defining a "esql" query that will make easier to see the results of a query, by generating a Pandas or Geopandas dataframe.

In [2]:
import os
import io

import warnings

from elasticsearch import Elasticsearch
from elasticsearch import ElasticsearchWarning
from elasticsearch.exceptions import BadRequestError

import pandas as pd
import geopandas as gpd
from shapely import wkb

# Hide the warning when no LIMIT is passed on a ES|QL query
warnings.filterwarnings('ignore', category=ElasticsearchWarning)

# Allow wide columns
pd.set_option('display.max_colwidth', None)

# Convert Well-known Binary to Text
def wkb_to_wkt(wkb_bytes):
    if wkb_bytes is None:
        return None
    try:
        return wkb.loads(wkb_bytes).wkt
    except Exception as e:
        print(f"Error converting WKB: {wkb_bytes} - {e}")
        return None

# Generate a Pandas Dataframe or a Geopandas Dataframe from a ES|QL query
def esql(query, geometry_col:str = "geometry", use_arrow:bool = False):
    try:
        # Query ES and create a Pandas Dataframe
        if use_arrow:
            es_response = client.esql.query(query=query.strip(), format="arrow", columnar=True)
            df = es_response.to_pandas()
        else:
            es_response = client.esql.query(query=query.strip(), format="csv")
            df = pd.read_csv(io.StringIO(str(es_response)))

        # Promote to a Geopandas Dataframe if a "geometry" column
        if geometry_col in df.columns:
            if use_arrow:
                # Arrow geometries are transferred as WKB
                df[geometry_col] = df[geometry_col].apply(wkb_to_wkt)
            gs = gpd.GeoSeries.from_wkt(df[geometry_col])
            gdf = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")
            if geometry_col != "geometry":
                gdf.drop(columns="geometry")
            return gdf
        else:
            return df
    except BadRequestError as e:
        print("Something went wrong!")
        print(e.message)
        print("\r\n".join([c['reason'] for c in e.info['error']['root_cause']]))

Connect to Elasticsearch and print some cluster details

In [3]:
# Login details
ES_URL=os.getenv("ES_URL","https://siglibre2025.es.us-central1.gcp.cloud.es.io")
KB_URL=os.getenv("KB_URL","https://siglibre2025.kb.us-central1.gcp.cloud.es.io")

# API key that allows reading indices
ES_APIKEY=os.getenv("ES_APIKEY", "NUF3RDlwZ0IxNDhsNzFFOXFDRWs6VS1xT1NKbTlGckdiMXdxVlBVd3RiQQ==")

# Load the client
client = Elasticsearch(hosts=[ES_URL], api_key=ES_APIKEY)

# Check the client
if client.ping():
  print("Connected to Elasticsearch")
  c_info = client.info()
  is_serverless = c_info['version']['build_flavor'] == 'serverless'

  # Print some cluster details
  print(f"Elasticsearch URL: {ES_URL}")
  print(f"Cluster name: {c_info['name']}")
  print(f"Version: {c_info['version']['number'] if not is_serverless else 'serverless'}")
  print("Number of documents indexed: ", client.count(index="*")['count'])
else:
  print("Connection failed")


Connected to Elasticsearch
Elasticsearch URL: https://siglibre2025.es.us-central1.gcp.cloud.es.io
Cluster name: instance-0000000004
Version: 9.1.3
Number of documents indexed:  524972


## Syntax and API

### Basic syntax

<https://www.elastic.co/docs/reference/query-languages/esql/esql-syntax>

A ES|QL query is made of a `source` command that sets the data to retrieve and a list of processing commands starting with the pipe `|` character.

```text
source-command
| processing-command1
| processing-command2
```

A query can contain one line and multi line comments.

```
source-command           // Single line comment
| processing-command1    // another comment
/*
a multi
line comment in between
processing commands
*/
| processing-command2
```

About literals:

* Literals are duble quoted
* If a double quote is required in a literal, triple quotes can be used

```text
ROW name = """Indiana "Indy" Jones"""
```


### Query API

* [Documentation](https://www.elastic.co/docs/explore-analyze/query-filter/languages/esql-rest)
* [Reference](https://www.elastic.co/docs/api/doc/elasticsearch/group/endpoint-esql)


Elasticsearch exposes the `_query` endpoint to execute ES|QL queries, with a `format` parameter to select between different output types as `csv`, `tsv`, `arrow`, `json`, etc.

As a `curl` command a request would be as:

```
curl -X POST \
  -H "Authorization: ApiKey $ES_APIKEY" \
  -H "Content-Type: application/json" \
  -d '{"query":"FROM places-* | STATS count = count(name) | LIMIT 1"}'\
  "$ES_URL/_query?pretty&format=txt"

     count     
---------------
230573
```

In the [Kibana Console](https://www.elastic.co/docs/explore-analyze/query-filter/tools/console):

```
POST /_query?format=txt
{
  "query": "FROM places-* | STATS count = count(name) | LIMIT 1"
}
```

And sending multiline queries is possible with triple quotes:

```text
POST /_query?format=txt
{
  "query": """
  FROM places-*
  | STATS count = count(name)
  | LIMIT 1
  """
}
```

A ES|QL API query can also include a filter using Elasticsearch DSL language:


```text
POST /_query?format=txt
{
  "query": """
  FROM places-*
  | STATS count = count(name)
  | LIMIT 1
  """,
  "filter": {
    "range": {
      "confidence": {
        "gte": 0.1,
        "lte": 1
      }
    }
  }
}
```

Other API endpoints available:

* [`_query/async`](https://www.elastic.co/docs/api/doc/elasticsearch/operation/operation-esql-async-query): start, stop, and get results asynchronously
* [`_query/queries`](https://www.elastic.co/docs/api/doc/elasticsearch/operation/operation-esql-list-queries): get details about running queries

## ES|QL sources: `ROW`, `SHOW`, `FROM`

In [4]:
# Creating a row directly, useful to test functions
esql('ROW a = 1, b = "two", c = null')

,a,b,c
0,1,two,NaN


In [5]:
# SHOW source returns Elasticsearch version
esql("SHOW INFO")

,version,date,hash
0,9.1.3,2025-08-24T22:05:04.526302670Z,0c781091a2f57de895a73a1391ff8426c0153c8d


In [6]:
# Basic query against all places indices
esql("FROM places-*")

,addresses.country,addresses.freeform,addresses.locality,addresses.postcode,addresses.region,alt_categories,brand,category,confidence,emails,geometry,id,name,phones,socials,source,updated,version,websites
0,ES,NaN,Granollers,NaN,NaN,stadium_arena,NaN,active_life,0.492940,NaN,POINT (2.3439 41.71782),afd644d9-9b38-4cf4-9591-1f2bc5a452f7,Pantano De Canoves,NaN,https://www.facebook.com/202670416579346,meta,2025-06-02T07:00:00.000Z,1,NaN
1,ES,NaN,NaN,NaN,NaN,NaN,NaN,structure_and_geography,0.492940,NaN,POINT (2.34202 41.71998),28d87f90-14e3-49f2-917a-a3f2c010324e,Embalse de Vallfornés,NaN,https://www.facebook.com/446907438814304,meta,2025-06-02T07:00:00.000Z,1,http://www.turisme-montseny.com/es/cultura/embalse-de-vallforners.html
2,ES,"Avinguda de Can Volart, 21",Cànoves i Samalús,8445.0,NaN,"[accommodation, restaurant]",NaN,hotel,0.978451,NaN,POINT (2.34867 41.72493),f910bd47-4ea1-499d-bf60-0c402054a22f,Hotel Can Cuch,3.493103e+10,https://www.facebook.com/169551469797250,meta,2025-06-02T07:00:00.000Z,1,http://www.hotelcancuch.com/
3,ES,Alto Bosque,Cánoves,NaN,NaN,"[fountain, topic_concert_venue]",NaN,landmark_and_historical_building,0.337662,NaN,POINT (2.36137 41.71496),eca6cb5e-4cc1-4af3-ab1c-ebfb5feb1480,El Bastión del Espino,3.465879e+10,https://www.facebook.com/1306002552764120,meta,2025-06-02T07:00:00.000Z,1,http://www.bastiondelespino.net/
4,ES,NaN,NaN,8445.0,NaN,"[catholic_church, social_service_organizations]",NaN,religious_organization,0.930061,NaN,POINT (2.35151 41.72199),7f061307-a767-46e4-a959-b87442196be1,Ermita de Sant Salvador,NaN,https://www.facebook.com/171874806222191,meta,2025-06-02T07:00:00.000Z,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,ES,"Carrer Pompeu Fabra, 17",Vidreres,17411.0,NaN,"[cafe, fast_food_restaurant]",NaN,coffee_shop,0.978451,NaN,POINT (2.77839 41.78937),1401d253-d824-4c39-98a3-406e57a9b53b,Cafè de la Selis,3.497285e+10,https://www.facebook.com/207867939348376,meta,2025-06-02T07:00:00.000Z,1,https://cafedelselis.sinergias360.biz
996,ES,"Carrer Pompeu Fabra, 14",Vidreres,17411.0,NaN,health_and_medical,NaN,beauty_and_spa,0.978451,NaN,POINT (2.77851 41.78938),8b7c6f71-dfe8-41df-9912-dd1e0a1e14f4,Thèna Centre de Bellesa i Benestar,3.469655e+10,https://www.facebook.com/105340174161300,meta,2025-06-02T07:00:00.000Z,1,NaN
997,ES,"Carrer Pompeu Fabra, 20",Vidreres,17411.0,NaN,"[fast_food_restaurant, restaurant]",NaN,italian_restaurant,0.978451,NaN,POINT (2.77824 41.78957),f6048570-77db-4cd6-a718-41ce5f3e8ec5,MOZZAFIATO cucina italiana,3.464076e+10,https://www.facebook.com/419908172127013,meta,2025-06-02T07:00:00.000Z,1,NaN
998,ES,"Carrer Pompeu Fabra, 13",Vidreres,17411.0,NaN,NaN,NaN,jewelry_store,0.930061,NaN,POINT (2.77852 41.78925),ea2a25b3-44fc-481a-84b5-eaa965ffbd1b,Joieria Comas,3.497285e+10,https://www.facebook.com/414341785741484,meta,2025-06-02T07:00:00.000Z,1,http://www.joieriacomas.com


## Control the output: `LIMIT`

By default a ES|QL query result is limited to `1000` rows. Use `LIMIT` to reduce that number

In [8]:
# Basic query against all places indices,
# returing the first 5 rows (in no particular order)

esql("""
FROM places-*
| LIMIT 3
""")

,addresses.country,addresses.freeform,addresses.locality,addresses.postcode,addresses.region,alt_categories,brand,category,confidence,emails,geometry,id,name,phones,socials,source,updated,version,websites
0,HR,Ulica Hrvoja Vukčića Hrvatinića 9,Split,21000,NaN,"[cafe, fast_food_restaurant]",NaN,coffee_shop,0.566292,NaN,POINT (16.44157 43.50883),ec75e7a3-e4ed-42c1-a6e0-8fbd5c7affe6,4coffee soul food,3.859768e+11,https://www.facebook.com/1541273576114848,meta,2025-06-02T07:00:00.000Z,1,NaN
1,HR,Hrvojeva 8,Split,21000,NaN,"[accommodation, holiday_rental_home]",NaN,hotel,0.492940,NaN,POINT (16.44167 43.50888),49a6e19f-5270-45a3-9de2-1064be7e7bec,In The Palace - Rooms,3.859159e+11,https://www.facebook.com/704101556286443,meta,2025-06-02T07:00:00.000Z,1,http://oglasnoslovno.wix.com/inthepalace
2,HR,Carrarina poljana 3,Split,21000,NaN,NaN,NaN,community_services_non_profits,0.337662,NaN,POINT (16.44106 43.50897),78f51e02-eb14-4775-a0a0-c72dcc018ea0,"Tae kwon-do klub ""Split""",NaN,https://www.facebook.com/404999039838455,meta,2025-06-02T07:00:00.000Z,1,NaN


## Change the output with `KEEP`, `RENAME`, and `SORT`

In [9]:
# Rename a field and only return a limited set of fields
esql("""
FROM places-*
| RENAME name as title
| KEEP title, category
| LIMIT 5
""")

,title,category
0,4coffee soul food,coffee_shop
1,In The Palace - Rooms,hotel
2,"Tae kwon-do klub ""Split""",community_services_non_profits
3,Splash,cafe
4,Tris Split,cafe


In [10]:
# KEEP also establishes the order of the columns returned,
# sometimes relevant for post-processing in client code
esql("""
FROM places-*
| RENAME name as title
| KEEP category, title
| LIMIT 5
""")

,category,title
0,coffee_shop,4coffee soul food
1,hotel,In The Palace - Rooms
2,community_services_non_profits,"Tae kwon-do klub ""Split"""
3,cafe,Splash
4,cafe,Tris Split


In [11]:
# Once renamed, the previous identifier is not available anymore
esql("""
FROM places-*
| RENAME name as title
| KEEP name, category
| LIMIT 5
""")

Something went wrong!
verification_exception
Found 1 problem
line 3:8: Unknown column [name]


In [12]:
# Sort by a field, and reverse another
esql("""
FROM places-girona
| RENAME name AS title
| SORT category ASC, title DESC
| KEEP category, title
| LIMIT 5
""")

,category,title
0,abuse_and_addiction_treatment,Uphill Centre d'Assessorament Terapèutic
1,abuse_and_addiction_treatment,Forum Salud Mental Girona
2,abuse_and_addiction_treatment,Día 1 Centre Terapèutic
3,accommodation,‘El racó del bandoler’
4,accommodation,Îles Medes


## Include metadata with `METADATA`

Use `METADATA` to get access to the `_index` and `_id`:

In [13]:
# Get also the source index using the METADATA keyword
esql("""
FROM places-* METADATA _index, _id
| KEEP _index, _id, name, category
| LIMIT 5
""")

,_index,_id,name,category
0,places-bosnia,ec75e7a3-e4ed-42c1-a6e0-8fbd5c7affe6,4coffee soul food,coffee_shop
1,places-bosnia,49a6e19f-5270-45a3-9de2-1064be7e7bec,In The Palace - Rooms,hotel
2,places-bosnia,78f51e02-eb14-4775-a0a0-c72dcc018ea0,"Tae kwon-do klub ""Split""",community_services_non_profits
3,places-bosnia,9064744a-1d2a-4ae8-86e0-bb84358c7bff,Splash,cafe
4,places-bosnia,584f66b1-ce42-4b07-8847-d1ccc3588ef8,Tris Split,cafe


## Filtering

In [16]:
# A basic filter
esql("""
FROM places-* METADATA _index
| RENAME _index as dataset
| WHERE name LIKE "*Burger*" //Exact text search
    AND category IN ("restaurant", "burger_restaurant") //Exact keyword search
    AND confidence < 0.3
| SORT confidence DESC
| KEEP dataset, name, category, confidence
| LIMIT 5
""")

,dataset,name,category,confidence
0,places-bosnia,Burgers by Manzoni,burger_restaurant,0.296943
1,places-valencia,TORO Burger Lounge,restaurant,0.296943
2,places-bosnia,Burger Factory-Igalo,burger_restaurant,0.245690
3,places-bosnia,Smash Burgers Two Guys,burger_restaurant,0.242152
4,places-bosnia,Fajnl Tač Burgers,burger_restaurant,0.242152


Truly leveraging Elasticsearch full text search features with the `MATCH` function.

[ES|QL documentation](https://www.elastic.co/docs/reference/query-languages/esql/functions-operators/search-functions#esql-match) | [DSL documentation](https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-query-string-query#query-string-syntax)

In [17]:
# Full text search with MATCH function
esql("""
FROM places-*
| WHERE MATCH(name, "snack BAR charlie", {"operator": "AND"})
| KEEP name
| LIMIT 1000
""")

,name
0,Charlie Bar and Snack


Sending queries in [Lucene](https://lucene.apache.org/) syntax with the `QSTR` function.

[ES|QL documentation](https://www.elastic.co/docs/reference/query-languages/esql/functions-operators/search-functions#esql-qstr) | [DSL documentation](https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-query-string-query) | [Query String syntax](https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-query-string-query#query-string-syntax)

In [18]:
# Full text search with Lucene syntax
# https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-query-string-query#query-string-syntax
esql("""
FROM places-* METADATA _index
| WHERE QSTR("name: CHARLIE snack bar",
            {"fuzziness": 2,
            "minimum_should_match": 2,
            "phrase_slop": 3})
| KEEP _index, name
| LIMIT 1000
""")

,_index,name
0,places-bosnia,Charlie Bar
1,places-bosnia,Caffe Bar Charlie Chaplin
2,places-bosnia,Charlie Bar and Snack
3,places-bosnia,Caffee bar Charlie
4,places-bosnia,Charlie Tango Lounge Bar


---
>For more examples on how to do different text search operations with ES|QL check this [tutorial](https://www.elastic.co/guide/en/elasticsearch/reference/current/esql-search-tutorial.html)
---

## Aggregations with `STATS`

In [19]:
# STATS allows running aggrecations.
# In this count agg, no other data is available afterwards
esql("""
FROM ne_countries
| STATS counts = count(id)
""")

,counts
0,257


In [20]:
# When grouping by other fields, those are also available
# for further operations like sorting or filtering
esql("""
FROM ne_countries
| WHERE type in ("Country", "Sovereign country")
| STATS counts = count(id) BY continent
| WHERE counts > 30
| SORT continent
| KEEP continent, counts //not necessary, only used to define order
| LIMIT 5
""")

,continent,counts
0,Africa,53
1,Asia,48
2,Europe,48


In [21]:
# Aggregate: count by more than one grouping field
esql("""
FROM ne_countries
| WHERE type not in ("Country", "Sovereign country")
| STATS counts = count(id) BY continent, type
| WHERE counts > 1
| SORT continent, type
| KEEP continent, type, counts
| LIMIT 50
""")

,continent,type,counts
0,Africa,Indeterminate,2
1,Asia,Dependency,3
2,Asia,Indeterminate,5
3,Europe,Disputed,2
4,North America,Dependency,12
5,North America,Indeterminate,2
6,Oceania,Dependency,12
7,Seven seas (open ocean),Dependency,5
8,South America,Indeterminate,2


## Computing new fields with `EVAL`

In [24]:
# Use EVAL to compute new fields
esql("""
FROM ne_countries
| WHERE gdp_md IS NOT NULL
    AND pop_est > 0
    AND type IN ("Country", "Sovereign country")
| EVAL gdp_pop = ROUND(gdp_md * 1e6 / ( pop_est::double))::integer
| SORT gdp_pop DESC
| KEEP name, type, gdp_md, pop_est, gdp_pop
| LIMIT 10
""")

,name,type,gdp_md,pop_est,gdp_pop
0,Monaco,Sovereign country,7188,38964,184478
1,Liechtenstein,Sovereign country,6876,38019,180857
2,Luxembourg,Sovereign country,71104,619896,114703
3,Isle of Man,Country,7491,84584,88563
4,Macao,Country,53859,640445,84096
5,Switzerland,Sovereign country,703082,8574832,81994
6,Ireland,Sovereign country,388698,4941444,78661
7,Norway,Sovereign country,403336,5347896,75420
8,Iceland,Sovereign country,24188,361313,66945
9,United States of America,Country,21433226,328239523,65298


## Joins

Traditionally, Elasticsearch has not been able to join datasets in a comfortable way for developers and analysts. This has changed with ES|QL and with the introduction of the `index.mode: lookup` setting.

More details on:

* [`LOOKUP JOIN` docs](https://www.elastic.co/docs/reference/query-languages/esql/esql-lookup-join)
* [`index.mode`](https://www.elastic.co/docs/reference/elasticsearch/index-settings/index-modules#index-mode-setting)

Our `ne_countries` index was created with that setting so we can now join that dataset with our places indices.

In [29]:
# Let's find outlier data in our places-* indices
esql(
"""
FROM places-*

// Aggregate all our places by country
| STATS counts = count(addresses.country) BY addresses.country

// Filter for only those groups with counts between 1 and 100
| WHERE counts > 1 AND counts < 10

// The lookup common field needs to be the same
| RENAME addresses.country AS iso_a2, counts AS places

// Run the LOOKUP JOIN by the iso_a2 field
| LOOKUP JOIN ne_countries ON iso_a2

// Merge records for repeated iso_a2 entries
| STATS places = MIN(places), names = VALUES(name) BY iso_a2

// Sort and print
| SORT places DESC
| KEEP iso_a2, places, names
"""
)

,iso_a2,places,names
0,MK,9,North Macedonia
1,RO,9,Romania
2,GB,7,United Kingdom
3,IN,6,India
4,AT,6,Austria
5,RU,6,Russia
6,DE,6,Germany
7,CH,6,Switzerland
8,VN,6,Vietnam
9,YE,5,Yemen


## Wrap up

This is just the basics, there are plenty of functions and operators to process your datasets, please refer to the [documentation](https://www.elastic.co/docs/explore-analyze/query-filter/languages/esql) for further details.